## Visualise stokes annulus model (flow etc)

In [1]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()
import os

In [2]:
import petsc4py
import underworld3 as uw
import numpy as np
import sympy

[Lyrebird.local:29776] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.Lyrebird.501/jf.0/1837039616/sm_segment.Lyrebird.501.6d7f0000.0 could be created.


In [3]:
ls -trl output/cartesian/ | tail

total 0
drwxr-xr-x@ 1409 lmoresi  staff  45088 Aug 19 21:32 Ra1e6.0/
drwxr-xr-x   601 lmoresi  staff  19232 Aug 20 05:19 Ra1e7.0/


In [4]:
ls -trl output/cartesian/Ra1e7.0/ | tail

-rw-r--r--   1 lmoresi  staff  724904 Aug 19 22:35 cartesian_Ra1e7.0_res20.mesh.T.00146.h5
-rw-r--r--   1 lmoresi  staff    3086 Aug 19 22:35 cartesian_Ra1e7.0_res20.mesh.00146.xdmf
-rw-r--r--   1 lmoresi  staff  412472 Aug 19 22:37 cartesian_Ra1e7.0_res20.mesh.U.00147.h5
-rw-r--r--   1 lmoresi  staff  135784 Aug 19 22:37 cartesian_Ra1e7.0_res20.mesh.P.00147.h5
-rw-r--r--   1 lmoresi  staff  724904 Aug 19 22:37 cartesian_Ra1e7.0_res20.mesh.T.00147.h5
-rw-r--r--   1 lmoresi  staff    3086 Aug 19 22:37 cartesian_Ra1e7.0_res20.mesh.00147.xdmf
-rw-r--r--   1 lmoresi  staff  412472 Aug 19 22:43 cartesian_Ra1e7.0_res20.mesh.U.00148.h5
-rw-r--r--   1 lmoresi  staff  135784 Aug 19 22:43 cartesian_Ra1e7.0_res20.mesh.P.00148.h5
-rw-r--r--   1 lmoresi  staff  724904 Aug 19 22:43 cartesian_Ra1e7.0_res20.mesh.T.00148.h5
-rw-r--r--   1 lmoresi  staff    3086 Aug 19 22:43 cartesian_Ra1e7.0_res20.mesh.00148.xdmf


In [5]:
geometry = "cartesian"
resolution = 20

checkpoint_dir = f"output/{geometry}/Ra1e7.0"
checkpoint_base = f"{geometry}_Ra1e7.0_res{resolution}"
meshfile = os.path.join(checkpoint_dir, checkpoint_base) + ".mesh.00000.h5"
image_dir = os.path.join(checkpoint_dir,"images")
os.makedirs(image_dir, exist_ok=True)

In [7]:
mesh = uw.discretisation.Mesh(meshfile, 
                              coordinate_system_type=uw.coordinates.CoordinateSystemType.CARTESIAN)
    
x = mesh.N.x
y = mesh.N.y

v_soln = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
t_soln = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)
flux = uw.discretisation.MeshVariable(r"dTdz", mesh, 1, degree=1)

In [23]:
steps = range(300,350,1)

In [24]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pl = pv.Plotter(window_size=[1000, 500])

for step in steps:

    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
        print(f"Plotting step {step}")

    except:
        print(f"Failed to read step {step} correctly")
        continue

    pl.clear()

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    # pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym[0])
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)

    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym)
    
    # point sources at cell centres
    skip = 10
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=True,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=False
    )

    pvstream = pvmesh.streamlines_from_source(
        point_cloud,
        vectors="V",
        integration_direction="both",
        max_time=1,
        surface_streamlines=True,
    )
    

    pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)    

    pl.camera_position = [(3.0, 0.5, 6.750994846396983),
                          (3.0, 0.5, 0.0),
                          (0.0, 1.0, 0.0)]

    imagefile = os.path.join(image_dir, checkpoint_base) + f"_V_T_{step}.png"
    
    pl.screenshot(filename=imagefile,
                  window_size=(1000, 500),
                  return_img=False)



Plotting step 300
Plotting step 301
Plotting step 302
Plotting step 303
Plotting step 304
Plotting step 305
Plotting step 306
Plotting step 307
Plotting step 308
Plotting step 309
Plotting step 310
Plotting step 311
Plotting step 312
Plotting step 313
Plotting step 314
Plotting step 315
Plotting step 316
Plotting step 317
Plotting step 318
Plotting step 319
Plotting step 320
Plotting step 321
Plotting step 322
Plotting step 323
Plotting step 324
Plotting step 325
Plotting step 326
Plotting step 327
Plotting step 328
Plotting step 329
Plotting step 330
Plotting step 331
Failed to read step 332 correctly
Failed to read step 333 correctly
Failed to read step 334 correctly
Failed to read step 335 correctly
Failed to read step 336 correctly
Failed to read step 337 correctly
Failed to read step 338 correctly
Failed to read step 339 correctly
Failed to read step 340 correctly
Failed to read step 341 correctly
Failed to read step 342 correctly
Failed to read step 343 correctly
Failed to read s

In [94]:
pl.show(jupyter_backend="trame")


Widget(value='<iframe src="http://localhost:62125/index.html?ui=P_0x3a33eb5d0_29&reconnect=auto" class="pyvist…

In [11]:
## Calculate heat flux, evaluate at surface — proxy for boundary layer thickness

In [12]:
t_soln.stats()

(12466,
 0.49898545293583557,
 2.3066821254869784e-06,
 0.9999978427445213,
 6220.352656298126,
 60.84989046004559,
 0.5449996702221376)

In [13]:
0/0

ZeroDivisionError: division by zero

In [ ]:
flux_solver = uw.systems.Projection(mesh, flux)

# Conductive flux only !
radial_flux = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_0)
radial_flux *= sympy.exp(-100*(r-1)**2) 

flux_solver.uw_function = radial_flux
flux_solver.smoothing = 1.0e-3
flux_solver.solve()

In [ ]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["dTdz"] = vis.scalar_fn_to_pv_points(pvmesh, flux.sym[0])
pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
pvmesh.point_data["V"] -= pvmesh.point_data["V"].mean()

velocity_points = vis.meshVariable_to_pv_cloud(v_soln)
velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, v_soln.sym)

# point sources at cell centres
skip = 8
points = np.zeros((mesh._centroids[::skip].shape[0], 3))
points[:, 0] = mesh._centroids[::skip, 0]
points[:, 1] = mesh._centroids[::skip, 1]
point_cloud = pv.PolyData(points)

pvstream = pvmesh.streamlines_from_source(
    point_cloud,
    vectors="V",
    integration_direction="both",
    max_time=0.5,
)

pl = pv.Plotter(window_size=[1000, 1000])

pl.add_mesh(
    pvmesh,
    cmap="Reds",
    edge_color="Black",
    show_edges=True,
    scalars="dTdz",
    use_transparency=False,
    opacity=1.0,
)

# 
# pl.add_mesh(pvmesh, "Black", "wireframe",  opacity=0.1)

# pl.add_points(point_cloud, color="White", point_size=3.0, opacity=0.25)

# pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.0001, show_scalar_bar=True)
pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)
    
# pl.remove_scalar_bar("V")

imagefile = os.path.join(image_dir, checkpoint_base) + f"_dTdr_{step}.png"

pl.screenshot(filename=imagefile, window_size=(750, 750), return_img=False)
# OR


In [ ]:
pl.show()